# Demo of the use of cosine distance between MPP encodings
This shows that the cosine distance between encodings roughly captures
whether they live in the same general part of the region.

## Package setup

In [ ]:
import numpy as np
import scipy
from geo_encodings import MPPEncoder

## Setup

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_array
from sklearn.cluster import DBSCAN
import shapely
import shapely.wkt
import plotly
import plotly.subplots
from plotly.graph_objs import Scatter, Figure

## Create an embedding for a region

In [ ]:
x0, y0 = 0, 0
x1, y1 = 200, 100
resolution = 20
scale = 10
encoder = MPPEncoder(region=[x0, y0, x1, y1], resolution=resolution, scale=scale, center=True)
print('%d reference points in encoder' % len(encoder))

## Create geometries

In [ ]:
# A polygon.
px = [10, 75, 80, 20, 10]
py = [10, 20, 80, 70, 10]
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(px, py))])
wkt = 'POLYGON((%s))' % xy
g1 = shapely.from_wkt(wkt)

# A multipoint. This is created so that it lives in the same general part of the region
# as the poygon that we just created.
n = 10
mpx = np.random.random(n) * 70 + 10
mpy = np.random.random(n) * 70 + 10
xy = ', '.join(['%.1f %.1f' % (z) for z in list(zip(mpx, mpy))])
wkt = 'MULTIPOINT(%s)' % xy
g2 = shapely.from_wkt(wkt)

In [ ]:
# Progressively shift the multipoint shape relative to the polygon.
# For each shifted position, re-compute its encoding and get the cosine distance 
# with the encoding for the polygon.
x_offsets = [0, 50, 100]
geoms = []
encodings = []
cosines = []

e1 = encoder.encode(g1).values()

for x_offset in [0, 50, 100]:
    g3 = shapely.affinity.translate(g2, xoff=x_offset)
    e3 = encoder.encode(g3).values()
    c = scipy.spatial.distance.cosine(e1, e3)
    geoms.append(g3)
    encodings.append(e3)
    cosines.append(c)


In [ ]:
fig = plotly.subplots.make_subplots(1, 1)

# Draw the reference points.
trace = Scatter(
    x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
    mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
)
fig.append_trace(trace, 1, 1)

# Draw the polygon.
coords = g1.exterior.coords
xx = [z[0] for z in coords]
yy = [z[1] for z in coords]
trace = Scatter(
    x=xx, y=yy, name='polygon', 
    mode='lines', marker={'color': 'red'}, fill='toself'
)
fig.append_trace(trace, 1, 1)

# Draw the multipoint.
xx = [z.xy[0][0] for z in g2.geoms]
yy = [z.xy[1][0] for z in g2.geoms]
trace = Scatter(
    x=xx, y=yy, name='multipoint', 
    mode='markers', marker={'color': 'red', 'size': 12}
)
fig.append_trace(trace, 1, 1)

fig['layout']['width'] = 700
fig['layout']['height'] = 400
fig['layout']['xaxis1']['range'] = [0, 200]
fig['layout']['yaxis1']['range'] = [0, 100]

fig

In [ ]:
titles = [
    'cosine distance = %.4f' % c
    for c in cosines
]
fig = plotly.subplots.make_subplots(3, 1, subplot_titles=titles)

for i in range(len(x_offsets)):

    irow = i + 1
    
    # Draw the reference points.
    trace = Scatter(
        x=encoder.ref_x, y=encoder.ref_y, name='reference points', 
        mode='markers', marker={'color': 'gray', 'size': 5, 'symbol': 'cross'}
    )
    fig.append_trace(trace, irow, 1)

    # Draw the polygon.
    coords = g1.exterior.coords
    xx = [z[0] for z in coords]
    yy = [z[1] for z in coords]
    trace = Scatter(
        x=xx, y=yy, name='polygon', 
        mode='lines', marker={'color': 'red'}, fill='toself'
    )
    fig.append_trace(trace, irow, 1)

    # Draw the multipoint.
    xx = [z.xy[0][0] for z in geoms[i].geoms]
    yy = [z.xy[1][0] for z in geoms[i].geoms]
    trace = Scatter(
        x=xx, y=yy, name='cos dist = %.3f' % c, 
        mode='markers', marker={'color': 'blue', 'size': 12}
    )
    fig.append_trace(trace, irow, 1)

fig['layout']['width'] = 700
fig['layout']['height'] = 900
fig['layout']['xaxis1']['range'] = [0, 200]
fig['layout']['yaxis1']['range'] = [0, 100]

fig

## Cosine distance between encodings

In [ ]:
e1 = encoder.encode(g1).values()
e2 = encoder.encode(g2).values()

In [ ]:
dd = []
cc = []

for dx in np.linspace(0, 100, 50):
    g3 = shapely.affinity.translate(g2, xoff=dx)
    e3 = encoder.encode(g3).values()
    c = scipy.spatial.distance.cosine(e1, e3)
    dd.append(dx)
    cc.append(c)

fig = plotly.subplots.make_subplots(1, 1)
trace = Scatter(x=dd, y=cc, mode='markers')
fig.append_trace(trace, 1, 1)
fig['layout']['xaxis1']['title'] = 'x offset'
fig['layout']['yaxis1']['title'] = 'cosine distance'
fig['layout']['width'] = 500
fig['layout']['height'] = 300

fig